In [2]:
from datasketch import MinHash, MinHashLSH
import string


In [3]:
class Deduplicate():
    def __init__(self, dataset, based_on, threshold=0.8, num_perm=128):
        self.dataset = dataset
        self.based_on = based_on # based on which category are we duduplicating the dataset
        self.threshold = threshold
        self.num_perm = num_perm

    # Function to preprocess text (tokenization into words)
    def preprocess_text(self, text):
        text = text.lower().translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
        return set(text.split())  # Convert to set of words

    # Function to create a MinHash signature for a text sample
    def create_minhash(self, text):
        minhash = MinHash(num_perm=self.num_perm)
        for word in self.preprocess_text(text):
            minhash.update(word.encode('utf8'))
        return minhash
    
    def deduplicate(self):
        # Define LSH to efficiently group similar MinHashes
        lsh = MinHashLSH(threshold=self.threshold, num_perm=self.num_perm)

        # Create a dictionary to map indices to MinHash signatures
        minhashes = {}
        for i, sample in enumerate(self.dataset):
            # some dataset may contain lists of dictionaries in sub-categories
            try: 
                text = sample[self.based_on]
                minhash = self.create_minhash(text)
            except: 
                text = sample[self.based_on][0]['input']
                minhash = self.create_minhash(text)
            
            lsh.insert(str(i), minhash)  # Insert into LSH
            minhashes[str(i)] = minhash

        # Find duplicates
        duplicates = set()
        for i, sample in enumerate(self.dataset):
            key = str(i)
            if key in duplicates:
                continue
            similar_items = lsh.query(minhashes[key])  # Get similar items
            if len(similar_items) > 1:
                duplicates.update(similar_items[1:])  # Mark duplicates

        # Create deduplicated dataset
        deduplicated_ds = [sample for i, sample in enumerate(self.dataset) if str(i) not in duplicates]

        # Display results
        print(f"Original dataset size: {len(self.dataset)}")
        print(f"Deduplicated dataset size: {len(deduplicated_ds)}")

        return deduplicated_ds
        

In [7]:
if __name__== "__main__":
    from datasets import load_dataset
    
    dataset = load_dataset("hkust-nlp/CodeIO-PyEdu-Reasoning", split="train", streaming=True)
    ds_duplicate = Deduplicate(dataset, based_on='prompt')
    ds_duplicate.deduplicate()

KeyboardInterrupt: 